In [1]:
# Basic packages
import pandas as pd 
import numpy as np
import re
import collections

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
NB_WORDS = 10000  # Parameter indicating the number of words we'll put in the dictionary

In [3]:
df = pd.read_csv('Tweets.csv')
df = df.reindex(np.random.permutation(df.index))  
df = df[['text', 'airline_sentiment']]
df.head()

,text,airline_sentiment
13360,"@AmericanAir @USAirways ""but I take meds that ...",negative
56,@VirginAmerica you know it. Need it on my spot...,positive
13232,@AmericanAir it's always nice coming home but ...,positive
8686,@JetBlue no news here at the gate. What are my...,negative
12269,@AmericanAir Lost and Found is like talking to...,negative


In [4]:
import nltk
nltk.download('stopwords')

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
       
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liyuantan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,airline_sentiment
13360,"""but take meds make severely dehydrated"" {sigh}",negative
56,know it. Need spotify stat #guiltypleasures,positive
13232,always nice coming home wish fly LAX-MAD keep...,positive
8686,no news gate. What options?,negative
12269,Lost Found like talking hormonal teenager ref...,negative


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.1, random_state=37)
print('# Train data samples:', X_train.shape[0])
print('# Test data samples:', X_test.shape[0])
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

# Train data samples: 13176
# Test data samples: 1464


In [6]:
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(X_train)

print('Fitted tokenizer on {} documents'.format(tk.document_count))
print('{} words in dictionary'.format(tk.num_words))
print('Top 5 most common words are:', collections.Counter(tk.word_counts).most_common(5))

Fitted tokenizer on 13176 documents
10000 words in dictionary
Top 5 most common words are: [('flight', 3538), ('not', 1435), ('no', 1350), ('get', 1184), ('t', 1101)]


In [7]:
X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

In [8]:
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

In [9]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(X_train_seq, maxlen=26)
labels = np.asarray(y_train_le)

In [10]:
training_samples = 10248  # We will be training on 2000 samples
validation_samples = 2928

In [11]:
# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = to_categorical(labels)

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(NB_WORDS, 32, input_length=26))
model.add(LSTM(32,
              dropout=0.2,
              recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 26, 32)            320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 328,419
Trainable params: 328,419
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(x_val, y_val))
model.save_weights('lstm_airline_validate.h5')

Train on 10248 samples, validate on 2928 samples
Epoch 1/10
10248/10248 [==============================] - 24s 2ms/step - loss: 0.7388 - acc: 0.6946 - val_loss: 0.6223 - val_acc: 0.7633
Epoch 2/10
10248/10248 [==============================] - 22s 2ms/step - loss: 0.5659 - acc: 0.7755 - val_loss: 0.5639 - val_acc: 0.7702
Epoch 3/10
10248/10248 [==============================] - 23s 2ms/step - loss: 0.5007 - acc: 0.8056 - val_loss: 0.5477 - val_acc: 0.7821
Epoch 4/10
10248/10248 [==============================] - 23s 2ms/step - loss: 0.4610 - acc: 0.8241 - val_loss: 0.5359 - val_acc: 0.7872
Epoch 5/10
10248/10248 [==============================] - 22s 2ms/step - loss: 0.4334 - acc: 0.8346 - val_loss: 0.5351 - val_acc: 0.7900
Epoch 6/10
10248/10248 [==============================] - 22s 2ms/step - loss: 0.4061 - acc: 0.8462 - val_loss: 0.5416 - val_acc: 0.7896
Epoch 7/10
10248/10248 [==============================] - 22s 2ms/step - loss: 0.3859 - acc: 0.8572 - val_loss: 0.5489 - val_acc: